In [1]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

from sklearn.model_selection import train_test_split 
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\Muhammad Ahsan\Desktop\PIAIC\CarPrice_Assignment.csv')
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [5]:
features = [col for col in df.columns.tolist() if col not in ['price','car_ID']]
numerical_features = []
categorical_features = [] 
target = df.price
dataset = df[features]

for col in features:
    if dataset[col].dtype in ['int64', 'int32', 'float64', 'float32']:
        numerical_features.append(col);
    else:
        categorical_features.append(col)
        
print('There are {:} numerical features.'.format(len(numerical_features)))
print('There are {:} categorical features.'.format(len(categorical_features)))

There are 14 numerical features.
There are 10 categorical features.


In [6]:
dataset.CarName.value_counts()

peugeot 504             6
toyota corona           6
toyota corolla          6
subaru dl               4
mitsubishi outlander    3
                       ..
dodge colt hardtop      1
vokswagen rabbit        1
subaru trezia           1
toyouta tercel          1
jaguar xk               1
Name: CarName, Length: 147, dtype: int64

In [7]:
dataset.loc[0,'CarName'].split(' ')[0]

'alfa-romero'

In [8]:
for item in range(len(dataset)):
    dataset.loc[item,'CarName'] = dataset.loc[item,'CarName'].split(' ')[0]
    
dataset[categorical_features].head()

C:\Users\Muhammad Ahsan\.conda\envs\tf-gpu\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,enginetype,cylindernumber,fuelsystem
0,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
1,alfa-romero,gas,std,two,convertible,rwd,front,dohc,four,mpfi
2,alfa-romero,gas,std,two,hatchback,rwd,front,ohcv,six,mpfi
3,audi,gas,std,four,sedan,fwd,front,ohc,four,mpfi
4,audi,gas,std,four,sedan,4wd,front,ohc,five,mpfi


In [9]:
dataset.CarName.value_counts()

toyota         31
nissan         17
mazda          15
mitsubishi     13
honda          13
subaru         12
peugeot        11
volvo          11
volkswagen      9
dodge           9
bmw             8
buick           8
audi            7
plymouth        7
saab            6
porsche         4
isuzu           4
alfa-romero     3
chevrolet       3
jaguar          3
maxda           2
vw              2
renault         2
porcshce        1
mercury         1
vokswagen       1
toyouta         1
Nissan          1
Name: CarName, dtype: int64

In [10]:
rare_name = ['isuzu','porsche','jaguar','chevrolet',
             'alfa-romero','maxda','vw','renault',
             'mercury','vokswagen','vokswagen','toyouta','Nissan','porcshce']

for item in range(len(dataset)):
    if dataset.loc[item,'CarName'] in rare_name:
        dataset.loc[item,'CarName'] = 'rare'
    else:
        pass

C:\Users\Muhammad Ahsan\.conda\envs\tf-gpu\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [11]:
dataset[categorical_features].describe().T

,count,unique,top,freq
CarName,205,16,toyota,31
fueltype,205,2,gas,185
aspiration,205,2,std,168
doornumber,205,2,four,115
carbody,205,5,sedan,96
drivewheel,205,3,fwd,120
enginelocation,205,2,front,202
enginetype,205,7,ohc,148
cylindernumber,205,7,four,159
fuelsystem,205,8,mpfi,94


In [12]:
dataset = pd.get_dummies(dataset, columns=categorical_features)
dataset.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
1,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,...,0,0,0,0,0,0,0,1,0,0
2,1,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,...,0,0,0,0,0,0,0,1,0,0
3,2,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,...,0,0,0,0,0,0,0,1,0,0
4,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
dataset['carvolume'] = dataset['carlength'] * dataset['carwidth'] * dataset['carheight']
dataset['totalmpg'] = dataset['citympg'] + dataset['highwaympg']
numerical_features.append('carvolume')
numerical_features.append('totalmpg')

In [14]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

categorical_features = [col for col in dataset.columns.tolist() if col not in numerical_features]

scaler = StandardScaler()
scaler = scaler.fit(dataset[numerical_features])
dataset_scaler = scaler.transform(dataset[numerical_features])
dataset_scaler = pd.DataFrame(dataset_scaler)
dataset_scaler.columns = numerical_features
dataset_scaler = pd.concat([dataset_scaler,dataset[categorical_features]], axis=1)
dataset_scaler.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,cylindernumber_twelve,cylindernumber_two,fuelsystem_1bbl,fuelsystem_2bbl,fuelsystem_4bbl,fuelsystem_idi,fuelsystem_mfi,fuelsystem_mpfi,fuelsystem_spdi,fuelsystem_spfi
0,1.743470,-1.690772,-0.426521,-0.844782,-2.020417,-0.014566,0.074449,0.519071,-1.839377,-0.288349,...,0,0,0,0,0,0,0,1,0,0
1,1.743470,-1.690772,-0.426521,-0.844782,-2.020417,-0.014566,0.074449,0.519071,-1.839377,-0.288349,...,0,0,0,0,0,0,0,1,0,0
2,0.133509,-0.708596,-0.231513,-0.190566,-0.543527,0.514882,0.604046,-2.404880,0.685946,-0.288349,...,0,0,0,0,0,0,0,1,0,0
3,0.938490,0.173698,0.207256,0.136542,0.235942,-0.420797,-0.431076,-0.517266,0.462183,-0.035973,...,0,0,0,0,0,0,0,1,0,0
4,0.938490,0.107110,0.207256,0.230001,0.235942,0.516807,0.218885,-0.517266,0.462183,-0.540725,...,0,0,0,0,0,0,0,1,0,0


In [15]:
features = dataset_scaler.values
features.shape 
target.shape

(205,)

In [16]:
# Applying Train Test Split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.29, random_state=42)

In [17]:
X_train.shape[1]

70

In [18]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(10, activation='tanh',input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(8, activation='tanh'))
    model.add(layers.Dense(6, activation='tanh'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [19]:
mdl = build_model()

In [21]:
mdl.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2,validation_data=(X_val,y_val))

Epoch 1/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 2/100
3/3 - 0s - loss: 209377232.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 3/100
3/3 - 0s - loss: 209377232.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 4/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 5/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 6/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 7/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 8/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 9/100
3/3 - 0s - loss: 209377200.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
E

Epoch 75/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 76/100
3/3 - 0s - loss: 209377200.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 77/100
3/3 - 0s - loss: 209377200.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 78/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 79/100
3/3 - 0s - loss: 209377200.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 80/100
3/3 - 0s - loss: 209377200.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 81/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 82/100
3/3 - 0s - loss: 209377232.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 14884.3213
Epoch 83/100
3/3 - 0s - loss: 209377216.0000 - mae: 12793.3809 - val_loss: 316060512.0000 - val_mae: 148

In [22]:
mdl.evaluate(X_test,y_test)

2/2 [==============================] - 0s 1ms/step - loss: 237550320.0000 - mae: 12971.7373


[237550320.0, 12971.7373046875]

In [23]:
y_test.iloc[1]

17859.167

In [100]:
import math

In [101]:
math.sqrt(16667995)

4082.6455883409717